![](https://raw.githubusercontent.com/interviewquery/takehomes/lyft_1/lyft_1/logo.png)
1. How do you define a driver that churned? What are the reasons? You should be able to assign `0` (not churn) or `1` (churn) in each unique driver. (100 words minimum)

2. How do you calculate churn rate based on your first answer of churn? The solution must be calculated with the available data in `driver_ids.csv`, `ride_ids.csv`, and `ride_timestamps.csv` (100 words minimum)

3. How to you cluster drivers into segments based on churn? (100 words minimum)

4. How would you reduce churn? Is there any effect on the business? (50 words minimum)

5. Come up with hypothesis to size the opportunity of reducing churn. Any effect on the business? (70 words minimum)


In [29]:
!git clone --branch lyft_1 https://github.com/interviewquery/takehomes.git
%cd takehomes/lyft_1
!ls

Cloning into 'takehomes'...
remote: Enumerating objects: 1963, done.
remote: Counting objects: 100% (1963/1963), done.
remote: Compressing objects: 100% (1220/1220), done.
remote: Total 1963 (delta 752), reused 1928 (delta 726), pack-reused 0
Receiving objects: 100% (1963/1963), 297.43 MiB | 16.37 MiB/s, done.
Resolving deltas: 100% (752/752), done.
Updating files: 100% (7/7), done.
/content/takehomes/lyft_1/takehomes/lyft_1
driver_ids.csv	logo.png  metadata.json  ride_ids.csv  ride_timestamps.csv  takehomefile.ipynb


In [30]:
#essential library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Dataset Loading and Overview

In [31]:
driverids_data = pd.read_csv('driver_ids.csv')
rideids_data = pd.read_csv('ride_ids.csv')
ridets_data = pd.read_csv('ride_timestamps.csv')

In [32]:
# Driver ID Dataset Info and Stats
print("The dataset has {} rows and {} columns".format(driverids_data.shape[0],driverids_data.shape[1]))
print(driverids_data.describe())
print(driverids_data.head())
print(driverids_data.dtypes.value_counts())
print("Dataset Columns:")
print(driverids_data.columns)

The dataset has 937 rows and 2 columns
                               driver_id  driver_onboard_date
count                                937                  937
unique                               937                   49
top     002be0ffdc997bd5c50703158b7c2491  2016-04-05 00:00:00
freq                                   1                   36
                          driver_id  driver_onboard_date
0  002be0ffdc997bd5c50703158b7c2491  2016-03-29 00:00:00
1  007f0389f9c7b03ef97098422f902e62  2016-03-29 00:00:00
2  011e5c5dfc5c2c92501b8b24d47509bc  2016-04-05 00:00:00
3  0152a2f305e71d26cc964f8d4411add9  2016-04-23 00:00:00
4  01674381af7edd264113d4e6ed55ecda  2016-04-29 00:00:00
object    2
dtype: int64
Dataset Columns:
Index(['driver_id', 'driver_onboard_date'], dtype='object')


In [33]:
# Ride ID Dataset Info and Stats
print("The dataset has {} rows and {} columns".format(rideids_data.shape[0],rideids_data.shape[1]))
print(rideids_data.describe())
print(rideids_data.head())
print(rideids_data.dtypes.value_counts())
print("Dataset Columns:")
print(rideids_data.columns)

The dataset has 193502 rows and 5 columns
       ride_distance  ride_duration  ride_prime_time
count  193502.000000  193502.000000    193502.000000
mean     6955.218266     858.966099        17.305893
std      8929.444606     571.375818        30.825800
min        -2.000000       2.000000         0.000000
25%      2459.000000     491.000000         0.000000
50%      4015.000000     727.000000         0.000000
75%      7193.000000    1069.000000        25.000000
max    724679.000000   28204.000000       500.000000
                          driver_id                           ride_id  \
0  002be0ffdc997bd5c50703158b7c2491  006d61cf7446e682f7bc50b0f8a5bea5   
1  002be0ffdc997bd5c50703158b7c2491  01b522c5c3a756fbdb12e95e87507eda   
2  002be0ffdc997bd5c50703158b7c2491  029227c4c2971ce69ff2274dc798ef43   
3  002be0ffdc997bd5c50703158b7c2491  034e861343a63ac3c18a9ceb1ce0ac69   
4  002be0ffdc997bd5c50703158b7c2491  034f2e614a2f9fc7f1c2f77647d1b981   

   ride_distance  ride_duration  ride_prim

In [34]:
# Ride Time Stamp Dataset Info and Stats
print("The dataset has {} rows and {} columns".format(ridets_data.shape[0],ridets_data.shape[1]))
print(ridets_data.describe())
print(ridets_data.head())
print(ridets_data.dtypes.value_counts())
print("Dataset Columns:")
print(ridets_data.columns)

The dataset has 970405 rows and 3 columns
                                 ride_id         event            timestamp
count                             970405        970405               970404
unique                            194081             5               865826
top     00003037a262d9ee40e61b5c0718f7f0  requested_at  2016-04-30 22:09:15
freq                                   5        194081                    7
                            ride_id           event            timestamp
0  00003037a262d9ee40e61b5c0718f7f0    requested_at  2016-06-13 09:39:19
1  00003037a262d9ee40e61b5c0718f7f0     accepted_at  2016-06-13 09:39:51
2  00003037a262d9ee40e61b5c0718f7f0      arrived_at  2016-06-13 09:44:31
3  00003037a262d9ee40e61b5c0718f7f0    picked_up_at  2016-06-13 09:44:33
4  00003037a262d9ee40e61b5c0718f7f0  dropped_off_at  2016-06-13 10:03:05
object    3
dtype: int64
Dataset Columns:
Index(['ride_id', 'event', 'timestamp'], dtype='object')


1. How do you define a driver that churned? What are the reasons? You should be able to assign `0` (not churn) or `1` (churn) in each unique driver. (100 words minimum)

Answer:

*   First of all it's important to see if the on-boarded driver has completed atleast one ride to categorize him as an activate driver.
*   Active drivers are categorized as churned drivers when they haven't completed a ride within a defined period, indicating disengagement.
*   Possible reasons might include low ride demand, dissatisfaction, or
external factors.
*   Assigning 0 or 1 involves analyzing ride frequency and gaps in activity, considering subjective factors like driver feedback.



In [35]:
# joining all the three datasets

merged_data = pd.merge(driverids_data, rideids_data, how='left', on='driver_id')
merged_data = pd.merge(merged_data, ridets_data, how='left', on='ride_id')
merged_data

,driver_id,driver_onboard_date,ride_id,ride_distance,ride_duration,ride_prime_time,event,timestamp
0,002be0ffdc997bd5c50703158b7c2491,2016-03-29 00:00:00,006d61cf7446e682f7bc50b0f8a5bea5,1811.0,327.0,50.0,requested_at,2016-04-23 02:13:50
1,002be0ffdc997bd5c50703158b7c2491,2016-03-29 00:00:00,006d61cf7446e682f7bc50b0f8a5bea5,1811.0,327.0,50.0,accepted_at,2016-04-23 02:14:15
2,002be0ffdc997bd5c50703158b7c2491,2016-03-29 00:00:00,006d61cf7446e682f7bc50b0f8a5bea5,1811.0,327.0,50.0,arrived_at,2016-04-23 02:16:36
3,002be0ffdc997bd5c50703158b7c2491,2016-03-29 00:00:00,006d61cf7446e682f7bc50b0f8a5bea5,1811.0,327.0,50.0,picked_up_at,2016-04-23 02:16:40
4,002be0ffdc997bd5c50703158b7c2491,2016-03-29 00:00:00,006d61cf7446e682f7bc50b0f8a5bea5,1811.0,327.0,50.0,dropped_off_at,2016-04-23 02:22:07
...,...,...,...,...,...,...,...,...
922805,ffff51a71f2f185ec5e97d59dbcd7a78,2016-05-04 00:00:00,ff7dc29693f8c79ff103d350a7b6c157,3751.0,889.0,100.0,requested_at,2016-05-15 21:08:47
922806,ffff51a71f2f185ec5e97d59dbcd7a78,2016-05-04 00:00:00,ff7dc29693f8c79ff103d350a7b6c157,3751.0,889.0,100.0,accepted_at,2016-05-15 21:08:55
922807,ffff51a71f2f185ec5e97d59dbcd7a78,2016-05-04 00:00:00,ff7dc29693f8c79ff103d350a7b6c157,3751.0,889.0,100.0,arrived_at,2016-05-15 21:11:26
922808,ffff51a71f2f185ec5e97d59dbcd7a78,2016-05-04 00:00:00,ff7dc29693f8c79ff103d350a7b6c157,3751.0,889.0,100.0,picked_up_at,2016-05-15 21:11:29


In [46]:
merged_data['event'].value_counts()

requested_at      184209
accepted_at       184209
arrived_at        184209
picked_up_at      184209
dropped_off_at    184209
Name: event, dtype: int64

In [47]:
# fix timedate formats

merged_data['timestamp'] = pd.to_datetime(merged_data['timestamp'])
merged_data['driver_onboard_date'] = pd.to_datetime(merged_data['driver_onboard_date'])

In [48]:
# last active date calculate using last occurence in the timestamp column for all unique driver ids

merged_data['last_active_date'] = merged_data.groupby('driver_id')['timestamp'].transform('max')


In [54]:
# analyze the merged dataset to label driver ids as churned 0 or 1 if they have been inactive for a specific timeframe - 7 days

merged_data['days_since_last_active'] = (merged_data['timestamp'].max() - merged_data['last_active_date']).dt.days
merged_data['is_churned'] = merged_data['days_since_last_active'] > 7


False    778855
True     143955
Name: is_churned, dtype: int64

In [61]:
# calculate churn rate

churn_counts = merged_data['is_churned'].value_counts()
churn_rate = (churn_counts[True]/churn_counts[False])*100
print("Churn Rate: {:.2f} %".format(churn_rate))

Churn Rate: 18.48 %


In [ ]:
# 2. How do you calculate churn rate based on your first answer of churn? The solution must be calculated with the available data in `driver_ids.csv`, `ride_ids.csv`, and `ride_timestamps.csv` (100 words minimum)
#
# Answer:
#
# Churn rate is calculated as the percentage of drivers who churned within a given time period.
#
# For example, if there are 1000 active drivers and 100 of them churned within a month, the churn rate would be 10%.
#
# To calculate churn rate using the available data, we can first identify the churned drivers by comparing the ride completion dates to the current date.
# Then, we can calculate the percentage of churned drivers relative to the total number of active drivers.
#
# Churn rate = (Number of churned drivers / Total number of active drivers) * 100
#
#
# 3. How do you cluster drivers into segments based on churn? (100 words minimum)
#
# Answer:
#
# Drivers can be clustered into segments based on their churn rate and other relevant factors.
# This can be done using clustering algorithms such as k-means clustering or hierarchical clustering.
#
# The goal of clustering is to identify groups of drivers with similar churn rates.
# This information can be used to understand the different reasons for churn and to develop targeted strategies to reduce churn.
#
#
# 4. How would you reduce churn? Is there any effect on the business? (50 words minimum)
#
# Answer:
#
# There are a number of things that can be done to reduce churn, including:
#
# *   Improving driver onboarding and training
# *   Providing better support for drivers
# *   Offering incentives to drivers to stay with the company
# *   Addressing the underlying causes of churn
#
# Reducing churn can have a positive impact on the business by increasing the number of active drivers and the amount of revenue generated.
#
#
# 5. Come up with hypothesis to size the opportunity of reducing churn. Any effect on the business? (70 words minimum)
#
# Answer:
#
# A hypothesis to size the opportunity of reducing churn could be that if the churn rate is reduced by 10%, the number of
# active drivers will increase by 5% and the revenue generated will increase by 10%.
#
# This hypothesis can be tested by conducting an experiment in which the churn rate is reduced by 10%.
# The results of the experiment can then be used to validate the hypothesis and to estimate the impact of reducing churn on the business.
